In [1]:
import pandas as pd
import json
import os
import glob
import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv("merged_vaccines.csv")

In [4]:
df.head(2)

,Tweet ID,Tweet Text,Type,Author Name,Author Username,Creation Time,Reply Count,Retweet Count,Quote Count,Like Count,...,Number of Likes,Number of Public Lists,Is Verified,Is Protected,Can DM,Can Tag in Media,Biography,User Homepage,Avatar URL,Profile Banner URL
0,"=""1807821100576895401""",The HPV vaccine is a critical step in preventi...,Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"7/1/2024, 5:58:32 PM",1.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"=""1807776904805580853""","New month, new experiences, new opportunities,...",Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"7/1/2024, 3:02:55 PM",0.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
import pandas as pd
import json
import os
import glob
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Define the folder paths on your local machine
folders = {
    'Twitter': r'C:\Users\hp\Desktop\scaped_data\twitter',
    'Facebook': r'C:\Users\hp\Desktop\scaped_data\facebook',
    'TikTok': r'C:\Users\hp\Desktop\scaped_data\Tiktok',
    'Instagram': r'C:\Users\hp\Desktop\scaped_data\Instagram',
    'Blogs': r'C:\Users\hp\Desktop\scaped_data\blogs'
}

# Column mapping dictionary - maps source columns to your target columns
column_mapping = {
    # Text content
    'text': 'Tweet Text', 'content': 'Tweet Text', 'caption': 'Tweet Text',
    'description': 'Tweet Text', 'post_text': 'Tweet Text', 'message': 'Tweet Text',
    'body': 'Tweet Text', 'title': 'Tweet Text', 'tweet': 'Tweet Text', 'post': 'Tweet Text',
    'full_text': 'Tweet Text', 'status': 'Tweet Text',
    
    # Author information
    'author': 'Author Name', 'user': 'Author Name', 'username': 'Author Username',
    'user_name': 'Author Username', 'handle': 'Author Username', 'author_username': 'Author Username',
    'name': 'Author Name', 'author_name': 'Author Name', 'user_handle': 'Author Username',
    'screen_name': 'Author Username', 'from': 'Author Name', 'from_name': 'Author Name',
    
    # Engagement metrics
    'likes': 'Like Count', 'like_count': 'Like Count', 'likes_count': 'Like Count',
    'favorite_count': 'Like Count', 'favourites': 'Like Count',
    'comments': 'Reply Count', 'comment_count': 'Reply Count', 'replies': 'Reply Count',
    'reply_count': 'Reply Count', 'shares': 'Retweet Count', 'share_count': 'Retweet Count',
    'retweets': 'Retweet Count', 'retweet_count': 'Retweet Count', 'views': 'View Count',
    'view_count': 'View Count', 'play_count': 'View Count', 'bookmarks': 'Bookmark Count',
    'bookmark_count': 'Bookmark Count', 'quotes': 'Quote Count', 'quote_count': 'Quote Count',
    
    # Timestamps
    'created_at': 'Creation Time', 'timestamp': 'Creation Time', 'date': 'Creation Time',
    'posted_at': 'Creation Time', 'publish_date': 'Creation Time', 'time': 'Creation Time',
    'createdat': 'Creation Time', 'created': 'Creation Time', 'posted': 'Creation Time',
    'published': 'Creation Time', 'post_date': 'Creation Time',
    
    # URLs and IDs
    'url': 'Tweet URL', 'post_url': 'Tweet URL', 'link': 'Tweet URL', 'tweeturl': 'Tweet URL',
    'permalink': 'Tweet URL', 'id': 'Tweet ID', 'post_id': 'Tweet ID', 'tweet_id': 'Tweet ID',
    'tweetid': 'Tweet ID', 'status_id': 'Tweet ID',
    
    # Other fields
    'hashtags': 'Hashtags', 'tags': 'Hashtags', 'language': 'Language', 'lang': 'Language',
    'location': 'Location', 'geo': 'Location', 'place': 'Location',
    'verified': 'Is Verified', 'is_verified': 'Is Verified',
    'followers': 'Number of Followers', 'follower_count': 'Number of Followers',
    'followers_count': 'Number of Followers',
    'following': 'Number of Following', 'following_count': 'Number of Following',
    'friends_count': 'Number of Following',
    'bio': 'Biography', 'biography': 'Biography',
    'media_type': 'Media Type', 'media_url': 'Media URLs', 'media_urls': 'Media URLs',
    'external_url': 'External URLs', 'external_urls': 'External URLs',
}

In [9]:
def read_file(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    
    try:
        if ext == '.csv':
            return pd.read_csv(file_path, low_memory=False, encoding='utf-8')
        elif ext == '.json':
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Handle both list of dicts and single dict
                if isinstance(data, list):
                    return pd.DataFrame(data)
                elif isinstance(data, dict):
                    # If it's a dict with a key containing list of posts
                    for key in ['posts', 'data', 'items', 'tweets', 'results', 'feed']:
                        if key in data and isinstance(data[key], list):
                            return pd.DataFrame(data[key])
                    return pd.DataFrame([data])
        elif ext in ['.xlsx', '.xls']:
            return pd.read_excel(file_path)
        elif ext == '.jsonl':
            data = []
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    if line.strip():
                        data.append(json.loads(line))
            return pd.DataFrame(data)
        else:
            print(f"      ⚠ Unsupported file type: {ext}")
            return None
    except Exception as e:
        print(f"      ✗ Error reading file: {str(e)}")
        return None

def standardize_columns(df_source, source_name):
    """Map source columns to target columns and add Source"""
    df_new = pd.DataFrame()
    
    # Convert all column names to lowercase for matching
    df_source.columns = df_source.columns.str.lower().str.strip().str.replace(' ', '_')
    
    # Map columns based on column_mapping
    for source_col, target_col in column_mapping.items():
        if source_col in df_source.columns:
            if target_col not in df_new.columns:  # Only add if not already added
                df_new[target_col] = df_source[source_col]
    
    # Copy any columns that already match target names (case-insensitive)
    target_cols_lower = {col.lower().replace(' ', '_'): col for col in df.columns}
    for col in df_source.columns:
        if col in target_cols_lower and target_cols_lower[col] not in df_new.columns:
            df_new[target_cols_lower[col]] = df_source[col]
    
    # Add Source column
    df_new['Source'] = source_name
    
    # Add Type column if not present
    if 'Type' not in df_new.columns:
        df_new['Type'] = source_name.lower() + '_post'
    
    return df_new

# Store all dataframes
all_dfs = [df]  # Start with your existing merged_vaccines.csv

print("=" * 70)
print("SOCIAL MEDIA DATA MERGER - LOCAL FILES")
print("=" * 70)
print(f"Starting with {len(df):,} rows from merged_vaccines.csv")
print(f"Columns in original data: {len(df.columns)}")
print("=" * 70)

total_files_processed = 0
total_rows_added = 0

# Process each folder
for source_name, folder_path in folders.items():
    print(f"\n{'=' * 70}")
    print(f"Processing {source_name.upper()} data...")
    print(f"{'=' * 70}")
    print(f"Path: {folder_path}")
    
    # Check if folder exists
    if not os.path.exists(folder_path):
        print(f"  ⚠ WARNING: Folder not found!")
        print(f"  Skipping {source_name}...")
        continue
    
    # Get all files in the folder (including subfolders)
    file_patterns = ['*.csv', '*.json', '*.xlsx', '*.xls', '*.jsonl']
    files = []
    for pattern in file_patterns:
        files.extend(glob.glob(os.path.join(folder_path, pattern)))
        files.extend(glob.glob(os.path.join(folder_path, '**', pattern), recursive=True))
    
    if not files:
        print(f"   No supported files found in {source_name} folder")
        continue
    
    print(f"  Found {len(files)} file(s)")
    
    # Process each file
    file_count = 0
    row_count = 0
    
    for file_path in files:
        file_name = os.path.basename(file_path)
        print(f"    📄 {file_name}...", end=" ")
        
        df_temp = read_file(file_path)
        
        if df_temp is not None and not df_temp.empty:
            # Standardize columns
            df_standardized = standardize_columns(df_temp, source_name)
            
            if not df_standardized.empty:
                all_dfs.append(df_standardized)
                file_count += 1
                row_count += len(df_standardized)
                total_files_processed += 1
                total_rows_added += len(df_standardized)
                print(f"✓ {len(df_standardized):,} rows added")
            else:
                print(f"⚠ No matching columns found")
        else:
            print(f"✗ Failed to read or empty file")
    
    print(f"\n   {source_name} Summary: {file_count} files processed, {row_count:,} rows added")

# Merge all dataframes
print(f"\n{'=' * 70}")
print("🔄 MERGING ALL DATA...")
print(f"{'=' * 70}")
print(f"Combining {len(all_dfs)} datasets...")

merged_df = pd.concat(all_dfs, ignore_index=True, sort=False)
print(f"✓ Total rows: {len(merged_df):,}")

# Skipping duplicate removal - you'll handle this yourself
print("\n✓ Keeping all rows (duplicates not removed)")

# Save the merged dataset
output_file = 'merged_vaccines_complete.csv'
merged_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"\n{'=' * 70}")
print(" MERGE COMPLETE!")
print(f"{'=' * 70}")
print(f" SUMMARY:")
print(f"  • Original dataset: {len(df):,} rows")
print(f"  • New data added: {total_rows_added:,} rows from {total_files_processed} files")
print(f"  • Total rows in merged file: {len(merged_df):,} rows")
print(f"  • Total columns: {len(merged_df.columns)}")
print(f"\n📍 Source Distribution:")
if 'Source' in merged_df.columns:
    source_counts = merged_df['Source'].value_counts()
    for source, count in source_counts.items():
        print(f"  • {source}: {count:,} posts")
else:
    print("  ⚠ Source column not found")

print(f"\n💾 Output saved to: {output_file}")
print(f"\n🔍 Preview of merged data:")
print(merged_df.head(10))
print(f"\n📋 All columns ({len(merged_df.columns)}):")
print(merged_df.columns.tolist())

# Display data types summary
print(f"\n📈 Data Summary:")
print(f"  • Memory usage: {merged_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
if 'Creation Time' in merged_df.columns:
    print(f"  • Date range: {merged_df['Creation Time'].min()} to {merged_df['Creation Time'].max()}")

print(f"\n{'=' * 70}")
print("🎉 ALL DONE! Your data is ready to use!")
print(f"{'=' * 70}")

SOCIAL MEDIA DATA MERGER - LOCAL FILES
Starting with 2,903 rows from merged_vaccines.csv
Columns in original data: 57

📁 Processing TWITTER data...
Path: C:\Users\hp\Desktop\scaped_data\twitter
  Found 322 file(s)
    📄 Lymphatic filariasis-twitter.csv... ✓ 10 rows added
    📄 soil transmitted helminths-twitter.csv... ✓ 10 rows added
    📄 TW_REPLY_amanpour_1369012679164256264_16 (1).csv... ✓ 16 rows added
    📄 TW_REPLY_amanpour_1369012679164256264_16.csv... ✓ 16 rows added
    📄 TW_REPLY_aproko_doctor_1192419710316531712_20 (1).csv... ✓ 20 rows added
    📄 TW_REPLY_aproko_doctor_1192419710316531712_20.csv... ✓ 20 rows added
    📄 TW_REPLY_aproko_doctor_1366388694899449859_100.csv... ✓ 100 rows added
    📄 TW_REPLY_aproko_doctor_1795370256560452065_79.csv... ✓ 79 rows added
    📄 TW_REPLY_aproko_doctor_1862025754688602169_100 (1).csv... ✓ 100 rows added
    📄 TW_REPLY_aproko_doctor_1862025754688602169_100 (2).csv... ✓ 100 rows added
    📄 TW_REPLY_aproko_doctor_1862025754688602169_100

TypeError: '<=' not supported between instances of 'str' and 'float'

In [12]:
merged_df.head()

,Tweet ID,Tweet Text,Type,Author Name,Author Username,Creation Time,Reply Count,Retweet Count,Quote Count,Like Count,...,Number of Likes,Number of Public Lists,Is Verified,Is Protected,Can DM,Can Tag in Media,Biography,User Homepage,Avatar URL,Profile Banner URL
0,"=""1807821100576895401""",The HPV vaccine is a critical step in preventi...,Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"7/1/2024, 5:58:32 PM",1.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"=""1807776904805580853""","New month, new experiences, new opportunities,...",Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"7/1/2024, 3:02:55 PM",0.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"=""1806318372742635560""",Blessing is protected against cervical cancer....,Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"6/27/2024, 2:27:14 PM",0.0,1.0,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"=""1806318348721954865""",https://t.co/wirqlPgG1q,Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"6/27/2024, 2:27:08 PM",0.0,0.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"=""1805543612580606082""",@science_nigeria @nighealthwatch @Fmohnigeria ...,Reply,Vaccine Network for Disease Control,VaccineNet_NG,"6/25/2024, 11:08:37 AM",0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
merged_df.shape

(48095, 57)

In [15]:
merged_df.to_csv("merged_vaccines_final.csv", index=False, encoding="utf-8-sig")

In [18]:
df3 = pd.read_csv("merged_vaccines_final.csv")

In [19]:
df3.head()

,Tweet ID,Tweet Text,Type,Author Name,Author Username,Creation Time,Reply Count,Retweet Count,Quote Count,Like Count,...,Number of Likes,Number of Public Lists,Is Verified,Is Protected,Can DM,Can Tag in Media,Biography,User Homepage,Avatar URL,Profile Banner URL
0,"=""1807821100576895401""",The HPV vaccine is a critical step in preventi...,Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"7/1/2024, 5:58:32 PM",1.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"=""1807776904805580853""","New month, new experiences, new opportunities,...",Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"7/1/2024, 3:02:55 PM",0.0,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"=""1806318372742635560""",Blessing is protected against cervical cancer....,Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"6/27/2024, 2:27:14 PM",0.0,1.0,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"=""1806318348721954865""",https://t.co/wirqlPgG1q,Tweet,Vaccine Network for Disease Control,VaccineNet_NG,"6/27/2024, 2:27:08 PM",0.0,0.0,0.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"=""1805543612580606082""",@science_nigeria @nighealthwatch @Fmohnigeria ...,Reply,Vaccine Network for Disease Control,VaccineNet_NG,"6/25/2024, 11:08:37 AM",0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df3.duplicated().sum()

np.int64(24928)